In [19]:
import pandas as pd
import openpyxl
import os
import re
import numpy as np

In [3]:
file_name = "/Users/luizacampos/Downloads/wbb_rosters_2022_23.xlsx"
file_path = os.path.join(os.getcwd(), file_name)
df= pd.read_excel(file_path, sheet_name="wbb_rosters_2022_23")


In [38]:
df.head()

,ncaa_id,team,player_id,name,year,hometown,homestate,high_school,previous_school_clean,height_clean,...,total_inches,primary_position,secondary_position,position_clean,year_clean,redshirt,hs_clean,hometown_clean,state_clean,country_clean
0,721,Air Force,11807.0,Mackenzie Le,Freshman,Elk Grove,California,St. Francis,NaN,"5'3""",...,63.0,GUARD,NaN,GUARD,Freshman,0,St. Francis,"ELK GROVE, CA",CA,USA
1,721,Air Force,11805.0,Milahnie Perry,Freshman,Tampa,Florida,Seffner Christian Academy,NaN,"5'7""",...,67.0,GUARD,NaN,GUARD,Freshman,0,Seffner Christian Academy,"TAMPA, FL",FL,USA
2,721,Air Force,11801.0,Madison Smith,Sophomore,Connell,Washington,Connell,NaN,"5'8""",...,68.0,GUARD,NaN,GUARD,Sophomore,0,Connell,"CONNELL, WA",WA,USA
3,721,Air Force,11795.0,Taylor Britt,Junior,Columbia,South Carolina,Spring Valley,USAFA Prep,"5'7""",...,67.0,GUARD,NaN,GUARD,Junior,0,Spring Valley,"COLUMBIA, SC",SC,USA
4,721,Air Force,11796.0,Kamri Heath,Senior,Edmond,Oklahoma,Edmond North,NaN,"5'7""",...,67.0,GUARD,NaN,GUARD,Senior,0,Edmond North,"EDMOND, OK",OK,USA


Primero mostramos en pantalla todas las columnas que contiene el dataframe. Eso es importante pues, a depender de que informaciones queremos obtener de los datos, podremos eliminar algunas. En el caso de este trabajo, vamos a evaluar la altura promedio de las jugadoras, la relacion entre su altura y la posición en que juegan, que equipos poseen más jugadoras extranjeras. 

In [39]:
print(df.columns)

Index(['ncaa_id', 'team', 'player_id', 'name', 'year', 'hometown', 'homestate',
       'high_school', 'previous_school_clean', 'height_clean', 'position',
       'jersey', 'url', 'season', 'team_state', 'conference', 'division',
       'height_ft', 'height_in', 'total_inches', 'primary_position',
       'secondary_position', 'position_clean', 'year_clean', 'redshirt',
       'hs_clean', 'hometown_clean', 'state_clean', 'country_clean'],
      dtype='object')


Pasamos a eliminar las columnas que no nos van a servir para el dashboard. Lo hicimos usando el metodo drop. Está por partes pues este método solo toma de 1 a 8 argumentos y queríamos eliminas muchas columnas.
Vamos a mantener la columna "team", que indica el equipo en que juegan, "name" que nos trae como información el nombre de cada jugadora a la que nos referimos, "height_clean" que nos indica la altura de la jugadora, "position_clean" que nos indica que posición principal juega y "country_clean" que nos informa el país de origen.

In [4]:
df = df.drop(['ncaa_id','player_id', 'year', 'hometown', 'homestate',
       'high_school', 'previous_school_clean'], axis=1)


In [5]:
df = df.drop([ 'url', 'season', 'team_state', 'conference', 'division',
       'height_ft', 'height_in', 'total_inches'], axis = 1)

In [6]:
df = df.drop(['year_clean', 'redshirt', 'jersey',
       'hs_clean', 'hometown_clean', 'state_clean'], axis=1)

In [ ]:
df = df.drop(['primary_position', 'secondary_position', 'position'], axis=1)

In [17]:
df.head()

,team,name,height_clean,position_clean,country_clean
0,Air Force,Mackenzie Le,"5'3""",GUARD,USA
1,Air Force,Milahnie Perry,"5'7""",GUARD,USA
2,Air Force,Madison Smith,"5'8""",GUARD,USA
3,Air Force,Taylor Britt,"5'7""",GUARD,USA
4,Air Force,Kamri Heath,"5'7""",GUARD,USA


En un segundo paso, averiguamos valores nulos. Lo vamos a hacer sumando la cantidad por columna e igualmente, averiguando el porcentaje de nulos en el dataframe

In [18]:
print(df.isnull().sum())

team               0
name               0
height_clean      39
position_clean    32
country_clean      0
dtype: int64


In [20]:
print('% de registros con al menos un valor nulo: ', np.round(100 * df.isna().any(axis=1).sum() / df.shape[0], 1))

% de registros con al menos un valor nulo:  0.5


Como se puede observar, los valores nulos representan el 0,5% de los registros. El dataframe posee un total de 13807 filas y en las columnas que usaremos, el valor de nulos es de 39 para "height_clean" y 32 para "position_clean". Por ende, considerando el número muy pequeño que no afectarían nuestros resultados, optamos por borrar los nulos. Eliminaremos tambien los duplicados

In [21]:
df = df.drop_duplicates()

In [22]:
df = df.dropna(how='any')


In [23]:
df.dropna(subset=['height_clean'], inplace=True)

In [27]:
df.tail(50)

,team,name,height_clean,position_clean,country_clean
13754,Belhaven,Jayda Jones,2023-06-05 00:00:00,GUARD,USA
13755,Belhaven,Kourtne Lee,2023-08-05 00:00:00,GUARD,USA
13756,Belhaven,Jayda Duncan,2023-08-05 00:00:00,GUARD,USA
13757,Belhaven,Allaijah Gamble,2023-10-05 00:00:00,GUARD,USA
13758,Belhaven,Ebony Gayden,2023-10-05 00:00:00,FORWARD,USA
13759,Belhaven,Hayle Poindexter,2023-02-06 00:00:00,FORWARD/CENTER,USA
13761,Belhaven,Ky'sha McNeely,2023-11-05 00:00:00,FORWARD/CENTER,USA
13763,SUNY Delhi,Aniya Jones,5'7,GUARD/FORWARD,USA
13764,SUNY Delhi,Justine Lubbers,5'9,FORWARD,USA
13765,SUNY Delhi,Yasmeen Kablaoui,5'2,GUARD,USA


In [25]:
print(df.isna().sum())

team              0
name              0
height_clean      0
position_clean    0
country_clean     0
dtype: int64


Si bien el dataframe ya no tiene nulos, evaluando el resultado, nos dimos cuenta que los datos cargados en la columna "height_clean" son de distintos tipos. Pasamos a averiguar 

In [29]:
tipo_de_datos = df['height_clean'].dtype
print(tipo_de_datos)

object


Usaremos una funcion para transformar los datos de la columna "height_clean". Esta función utiliza expresiones regulares para buscar patrones en las cadenas de altura y convertirlos a centímetros. Los patrones son los siguientes:
feet_inches_pattern: busca un número seguido de un carácter que indica pies o pulgadas, luego otro número y otro carácter de pies o pulgadas. Ejemplos: "6'11", "5-8".
feet_pattern: busca un número seguido de un carácter que indica pies. Ejemplo: "6'0".
inches_pattern: busca un carácter que indica pulgadas seguido de un número. Ejemplo: "5'9".
Si la cadena de entrada no coincide con ninguno de estos patrones, la función devuelve None.

In [31]:
def height_to_cm(height_str):
    feet_inches_pattern = re.compile(r'(\d+)([\W_])(\d+)')
    feet_pattern = re.compile(r'(\d+)([\W_])')
    inches_pattern = re.compile(r'([\W_])(\d+)')
    
    match = feet_inches_pattern.match(height_str)
    if match:
        feet = int(match.group(1))
        inches = int(match.group(3))
    else:
        match = feet_pattern.match(height_str)
        if match:
            feet = int(match.group(1))
            inches = 0
        else:
            match = inches_pattern.search(height_str)
            if match:
                feet = 0
                inches = int(match.group(2))
            else:
                return None
    
    total_inches = feet * 12 + inches
    height_cm = total_inches * 2.54
    return height_cm

In [33]:
df['height_clean'] = df['height_clean'].astype(str)

Vamos a crear una nueva columna llamando la función para transformar la data de height_clean a sistema métrico

In [34]:
df['height_cm'] = df['height_clean'].apply(height_to_cm)

In [36]:
df.head(50)

,team,name,height_clean,position_clean,country_clean,height_cm
0,Air Force,Mackenzie Le,"5'3""",GUARD,USA,160.02
1,Air Force,Milahnie Perry,"5'7""",GUARD,USA,170.18
2,Air Force,Madison Smith,"5'8""",GUARD,USA,172.72
3,Air Force,Taylor Britt,"5'7""",GUARD,USA,170.18
4,Air Force,Kamri Heath,"5'7""",GUARD,USA,170.18
5,Air Force,Kayla Pilson,"5'10""",GUARD/FORWARD,USA,177.80
6,Air Force,Faith Shelton,"6'2""",FORWARD/CENTER,USA,187.96
7,Air Force,Griffin Greenwood,"5'11""",FORWARD,USA,180.34
8,Air Force,Parker Brown,"6'0""",FORWARD,USA,182.88
9,Air Force,Dasha Macmillan,"6'0""",FORWARD,USA,182.88


Finalmente, salvamos el archivo para usarlo en Tableau

In [52]:
df.to_excel(file_path, index=False)